<h1>Training</h1>

In [1]:
model_storage = "D:/MLDatasets/TCGA-GBM Datasets/Models"

In [ ]:
from dataloadingv1 import dataloader

<h3>UNET Definition</h3>

In [3]:
import os
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.functional as F
from monai.networks.nets import UNet
from monai.networks.layers import Norm 
from monai.losses import DiceLoss

from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split


class UNETTransfer(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = UNet(
            dimensions=3,
            in_channels=1,
            out_channels=2,
            channels=(16, 32, 64, 128, 256), 
            strides=(2, 2, 2, 2),
            num_res_units=2,
            norm=Norm.BATCH
        )
        self.loss = DiceLoss(to_onehot_y=True, sigmoid=True, squared_pred=True)

    def training_step(self, batch, batch_idx):
        vol, seg = batch['vol'], batch['seg']
        out = self.model(vol)
        res_loss = self.loss(out, vol)
        self.log('train loss',  res_loss)
        return res_loss 
     
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), 1e-5, weight_decay=1e-5, amsgrad=True)

test_model = UNETTransfer()
print(test_model.model)

UNet(
  (model): Sequential(
    (0): ResidualUnit(
      (conv): Sequential(
        (unit0): Convolution(
          (conv): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
          (adn): ADN(
            (N): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (D): Dropout(p=0.0, inplace=False)
            (A): PReLU(num_parameters=1)
          )
        )
        (unit1): Convolution(
          (conv): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
          (adn): ADN(
            (N): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (D): Dropout(p=0.0, inplace=False)
            (A): PReLU(num_parameters=1)
          )
        )
      )
      (residual): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    )
    (1): SkipConnection(
      (submodule): Sequential(
        (0): ResidualUnit(
          (conv): Sequential(


In [4]:
print(f"Trainloader Length: {len(trainloader)}")

Trainloader Length: 102


In [ ]:
torch.cuda.empty_cache()

In [ ]:
medel = SegResNetTransfer()

'''SET TRAINING EPOCHS'''
epochs = 10 

#last_checkpoint = './lightning_logs/version_9/checkpoints/epoch=4-step=510.ckpt'
trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=epochs)
trainer.fit(model=model, train_dataloaders=trainloader) # ckpt_path=last_checkpoint)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | model | SegResNet | 1.2 M 
1 | loss  | DiceLoss  | 0     
------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.705     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
